In [2]:
import numpy as np
from pathlib import Path
from collections import defaultdict as ddict
from collections import OrderedDict as odict
from easydict import EasyDict as edict
import pandas as pd

### 1. transform dk data with new labels

In [3]:
def labNew2Old(string):
    prefix = "ctx-"
    itm1, itm2 = string.split(".")
    return prefix + itm1 + "-" + itm2

In [6]:
filDir = Path("../data/brain_plot_meg/")

fcROld = pd.read_csv(filDir/'DK_dictionary_normalized.csv').set_index('Unnamed: 0')
dkLabs = pd.read_excel(filDir/'DK_labs_list.xlsx')
fcRarr = np.array(fcROld)

oldLabs = list(fcROld.columns)
dkLabs = list(np.array(dkLabs).reshape(-1))

In [25]:
fcRNewDic = odict()

for lab in dkLabs:
    oldlab = labNew2Old(lab)
    fcRNewDic[oldlab] = list(fcROld[oldlab])
    
fcRNew = pd.DataFrame(fcRNewDic)
fcRNew.index = fcROld.index
fcRNew.to_csv(filDir/"DK_dic68.csv")

### 2. Obtain the U, weighted U and corresponding correlation for fMRI data

In [11]:
import sys
sys.path.append("/home/huaqingj/MyResearch/TVDN")
from pyTVDN.tmpUtils import *
import pickle
import scipy.stats as ss

#### Get the fcR

In [12]:
fcRDF = pd.read_csv("../necessary files/AALICA.csv")
fcR = np.array(fcRDF)[:90, :]
names = list(fcRDF.columns)
names = [name.strip() for name in names]

fcR = np.array([minmax(fcR[:, i]) for i in range(7)])
fcR = fcR.T

#### U 

In [13]:
datDir = Path("../results/fMRIHPFs_rankAdap")
fil = list(datDir.glob("*AdapM8.pkl"))[0]
print(fil)

with open(fil, "rb") as f:
    TVDNres = pickle.load(f)
    
Us = []
for DetObj in TVDNres.DetObjs:
    r = DetObj.paras.r
    rUs = DetObj.midRes.eigVecs[:, :r]
    kpidxs = np.where(np.diff(np.concatenate([[np.Inf], np.abs(rUs[0, :])])) != 0)[0]
    for ix in kpidxs:
        Us.append(rUs[:, ix])
        
absUs = np.array(np.abs(Us))
absUsList =[minmax(absUs[i, :]) for i in range(absUs.shape[0])]
absUs = np.array(absUsList)

../results/fMRIHPFs_rankAdap/fMRIResultAdapM8.pkl


#### correlation

In [14]:
corrs = ddict(list)
for absU in absUs:
    for j, name in enumerate(names):
        corrs[name].append(ss.pearsonr(fcR[:, j], absU)[0])
        
corDF = pd.DataFrame(corrs)
corArr = np.array(corDF)

#np.savetxt(datDir/"allCorrUM8.txt", corArr)

### Correlation  of demo 

In [62]:
idx = 128
DetObj = TVDNres.DetObjs[idx]
r = DetObj.paras.r
rUs = DetObj.midRes.eigVecs[:, :r]
kpidxs = np.where(np.diff(np.concatenate([[np.Inf], np.abs(rUs[0, :])])) != 0)[0]
Us = rUs[:, kpidxs]

absUs = np.array(np.abs(Us)).T
absUsList =[minmax(absUs[i, :]) for i in range(absUs.shape[0])]
absUs = np.array(absUsList)


abswUs = obtainAbswU(DetObj)
abswUsList = [minmax(abswUs[:, i]) for i in range(abswUs.shape[1])]
abswUs = np.array(abswUsList)

abswUs2 = obtainAbswU2(DetObj)
abswUsList2 = [minmax(abswUs2[:, i]) for i in range(abswUs2.shape[1])]
abswUs2 = np.array(abswUsList2)

#### Corr of U

In [63]:
corrs = ddict(list)
for absU in absUs:
    for j, name in enumerate(names):
        corrs[name].append(ss.pearsonr(fcR[:, j], absU)[0])
        
corDF = pd.DataFrame(corrs)
corDF

,Visual,Somatomotor,Dorsal_Attention,Ventral_Attention,Limbic,Frontoparietal,Default
0,0.064560,-0.172343,0.351655,0.004877,0.042928,0.217249,0.064095
1,-0.116244,0.027635,-0.009819,-0.053607,0.263776,0.022295,0.114331
2,-0.081011,-0.183046,-0.005128,-0.013957,0.414429,0.130746,-0.063333
3,0.138192,0.043554,-0.040197,0.019383,0.014288,0.155347,-0.084598
4,0.109612,-0.026601,0.086831,-0.043546,0.135752,-0.003456,-0.037758


#### Corr of wU

In [64]:
wUcorrs = ddict(list)
for abswU in abswUs:
    for j, name in enumerate(names):
        wUcorrs[name].append(ss.pearsonr(fcR[:, j], abswU)[0])
        
wUcorDF = pd.DataFrame(wUcorrs)
wUcorDF

,Visual,Somatomotor,Dorsal_Attention,Ventral_Attention,Limbic,Frontoparietal,Default
0,-0.082289,-0.050062,-0.150108,0.229821,0.067254,0.012267,0.160855
1,-0.198210,0.012349,-0.141684,0.069154,0.102017,-0.008767,0.093619
2,-0.174588,-0.151454,-0.080028,0.049320,0.380066,0.035140,0.143097
3,-0.113006,0.147928,-0.173036,0.030069,-0.049907,-0.069072,-0.111928
4,-0.049861,-0.025812,0.054267,-0.100846,0.215028,0.026206,0.075160
5,0.100392,0.197857,0.000866,-0.154123,-0.159477,-0.076182,-0.106719
6,-0.014538,0.103572,0.047158,0.021729,0.049448,0.003455,-0.086708


#### Corr of wU via second way

In [65]:
wUcorrs2 = ddict(list)
for abswU in abswUs2:
    for j, name in enumerate(names):
        wUcorrs2[name].append(ss.pearsonr(fcR[:, j], abswU)[0])
        
wUcorDF2 = pd.DataFrame(wUcorrs2)
wUcorDF2

,Visual,Somatomotor,Dorsal_Attention,Ventral_Attention,Limbic,Frontoparietal,Default
0,0.045672,-0.051739,0.134075,-0.024167,0.185808,0.158026,0.032018
1,0.003277,-0.077004,0.189048,-0.029778,0.183930,0.142884,0.078881
2,-0.006887,-0.140640,0.186414,-0.022522,0.255633,0.172744,0.038864
3,0.022899,-0.076907,0.153440,-0.022035,0.204205,0.172693,0.043315
4,0.029050,-0.078258,0.134556,-0.020580,0.218849,0.176510,0.025985
5,0.035884,-0.064112,0.151318,-0.024476,0.190055,0.160950,0.041237
6,0.072401,-0.018299,0.090121,-0.018463,0.156574,0.158398,0.008303


In [66]:
idx = 212
DetObj = TVDNres.DetObjs[idx]
r = DetObj.paras.r
rUs = DetObj.midRes.eigVecs[:, :r]
kpidxs = np.where(np.diff(np.concatenate([[np.Inf], np.abs(rUs[0, :])])) != 0)[0]
Us = rUs[:, kpidxs]

absUs = np.array(np.abs(Us)).T
absUsList =[minmax(absUs[i, :]) for i in range(absUs.shape[0])]
absUs = np.array(absUsList)

abswUs = obtainAbswU(DetObj)
abswUsList = [minmax(abswUs[:, i]) for i in range(abswUs.shape[1])]
abswUs = np.array(abswUsList)

abswUs2 = obtainAbswU2(DetObj)
abswUsList2 = [minmax(abswUs2[:, i]) for i in range(abswUs2.shape[1])]
abswUs2 = np.array(abswUsList2)

#### Corr of U

In [67]:
corrs = ddict(list)
for absU in absUs:
    for j, name in enumerate(names):
        corrs[name].append(ss.pearsonr(fcR[:, j], absU)[0])
        
corDF = pd.DataFrame(corrs)
corDF
#np.savetxt(datDir/"allCorrUM8.txt", corArr)

,Visual,Somatomotor,Dorsal_Attention,Ventral_Attention,Limbic,Frontoparietal,Default
0,0.034287,0.014605,0.081091,0.035885,0.154508,0.154234,-0.149754
1,-0.072332,0.009480,0.154516,-0.106884,0.018489,0.010186,-0.192265
2,0.149625,0.085701,-0.159209,0.274301,-0.071149,0.061180,-0.325346
3,0.095383,-0.258477,0.013012,-0.116173,0.300563,0.285218,-0.095110


#### Corr of wU

In [68]:
wUcorrs = ddict(list)
for abswU in abswUs:
    for j, name in enumerate(names):
        wUcorrs[name].append(ss.pearsonr(fcR[:, j], abswU)[0])
        
wUcorDF = pd.DataFrame(wUcorrs)
wUcorDF

,Visual,Somatomotor,Dorsal_Attention,Ventral_Attention,Limbic,Frontoparietal,Default
0,0.122207,-0.043322,0.148418,0.176174,-0.140782,0.063986,-0.261804
1,0.353833,-0.071972,-0.116174,-0.084834,0.090960,0.112339,-0.201380
2,-0.051349,0.160612,0.241598,0.134585,-0.074321,0.029233,-0.069004
3,-0.100899,-0.090253,0.027775,0.156608,0.017129,-0.022718,-0.051951
4,0.028979,-0.013940,-0.126298,0.003308,0.353886,0.052833,-0.042483
5,-0.092616,-0.003557,0.081098,0.055203,0.071068,0.185249,-0.126507
6,0.226509,0.067102,-0.120653,0.040738,0.056613,-0.000330,-0.158481
7,0.224425,-0.005062,0.150190,-0.055697,-0.041002,0.091291,-0.126319
8,0.130095,-0.217796,0.061923,0.082134,0.102055,-0.013960,-0.042700


#### Corr of wU via second way

In [69]:
wUcorrs2 = ddict(list)
for abswU in abswUs2:
    for j, name in enumerate(names):
        wUcorrs2[name].append(ss.pearsonr(fcR[:, j], abswU)[0])
        
wUcorDF2 = pd.DataFrame(wUcorrs2)
wUcorDF2

,Visual,Somatomotor,Dorsal_Attention,Ventral_Attention,Limbic,Frontoparietal,Default
0,0.121198,-0.059566,-0.037662,0.093181,0.128515,0.196265,-0.300009
1,0.076953,-0.025452,0.036148,0.057674,0.142947,0.184996,-0.285314
2,0.096810,-0.039344,0.012495,0.073230,0.156345,0.203174,-0.279287
3,0.039639,0.015546,0.075226,0.042584,0.111343,0.142634,-0.274703
4,0.050891,-0.045738,0.076753,0.002557,0.168807,0.188632,-0.256136
5,0.060911,-0.020601,0.048251,0.040766,0.119048,0.161990,-0.300384
6,0.078098,-0.009462,0.024843,0.074679,0.112468,0.165858,-0.308510
7,0.083745,-0.071730,0.038004,0.025464,0.187744,0.218326,-0.259859
8,0.062451,-0.069584,0.053725,-0.000623,0.157370,0.188684,-0.278874
